In [ ]:
import pandas as pd
import numpy as np
from itertools import compress
from sklearn.metrics import cohen_kappa_score,f1_score,accuracy_score, precision_score, recall_score, classification_report, roc_auc_score, \
    hamming_loss


# [TODO] edit path to match experiments outputs
SOURCE_PATH_embeds = "/src_code/repo/experiments_results/single_label_deeper_weighted_normalized_long/2020-05-09_11-44-38"
gt = pd.read_csv(SOURCE_PATH_embeds + "/groundtruth_withIDS.csv")
predu = pd.read_csv(SOURCE_PATH_embeds + '/probabilities_withIDS.csv')
one_hot_u = pd.read_csv(SOURCE_PATH_embeds + '/onehot_withIDS.csv')
SOURCE_PATH_auds = "/src_code/repo/experiments_results/classic_updated_dataset_long/2020-05-08_13-17-43"
one_hot_a = pd.read_csv(SOURCE_PATH_auds + '/one_hoted_withIDS.csv')

In [ ]:
# create a frame of NAs 
LABELS_LIST = ['car', 'gym', 'happy', 'night', 'relax',
    'running', 'sad', 'summer', 'work', 'workout']
users_audio_f1 = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_embeds_f1 = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_audio_recall = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_embeds_recall = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_audio_acc = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_embeds_acc = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_audio_prec = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_embeds_prec = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
for x in LABELS_LIST:
    users_audio_f1[x] = np.NAN
    users_embeds_f1[x] = np.NAN
    users_audio_recall[x] = np.NAN
    users_embeds_recall[x] = np.NAN
    users_audio_acc[x] = np.NAN
    users_embeds_acc[x] = np.NAN
    users_audio_prec[x] = np.NAN
    users_embeds_prec[x] = np.NAN
users_audio_f1 = users_audio_f1.set_index("user_id")
users_embeds_f1 = users_embeds_f1.set_index("user_id")
users_audio_recall = users_audio_recall.set_index("user_id")
users_embeds_recall = users_embeds_recall.set_index("user_id")
users_audio_acc = users_audio_acc.set_index("user_id")
users_embeds_acc = users_embeds_acc.set_index("user_id")
users_audio_prec = users_audio_prec.set_index("user_id")
users_embeds_prec = users_embeds_prec.set_index("user_id")

# compute scores for only present labels
for user in users_audio_f1.index:
    user_truth = gt[gt.user_id == user]
    user_preds_audio = one_hot_a.loc[preda.user_id == user]
    user_preds_embeds = one_hot_u.loc[predu.user_id == user]
    active_labels_idx = user_truth.sum() > 0
    active_labels_idx = active_labels_idx[2:]
    active_labels = list(compress(LABELS_LIST, active_labels_idx))
    for label in active_labels:
        users_audio_f1.loc[user][label] = f1_score(user_truth[label],user_preds_audio[label])
        users_embeds_f1.loc[user][label] = f1_score(user_truth[label],user_preds_embeds[label])
        users_audio_recall.loc[user][label] = recall_score(user_truth[label],user_preds_audio[label])
        users_embeds_recall.loc[user][label] = recall_score(user_truth[label],user_preds_embeds[label])
        users_audio_acc.loc[user][label] = accuracy_score(user_truth[label],user_preds_audio[label])
        users_embeds_acc.loc[user][label] = accuracy_score(user_truth[label],user_preds_embeds[label])
        users_audio_prec.loc[user][label] = precision_score(user_truth[label],user_preds_audio[label])
        users_embeds_prec.loc[user][label] = precision_score(user_truth[label],user_preds_embeds[label])
        
# averaging per user
user_audio_f1_mean = users_audio_f1.mean(axis=1)
user_embeds_f1_mean = users_embeds_f1.mean(axis=1)

users_audio_recall_mean = users_audio_recall.mean(axis=1)
user_embeds_recall_mean = users_embeds_recall.mean(axis=1)

user_audio_acc_mean = users_audio_acc.mean(axis=1)
user_embeds_acc_mean = users_embeds_acc.mean(axis=1)

user_audio_prec_mean = users_audio_prec.mean(axis=1)
user_embeds_prec_mean = users_embeds_prec.mean(axis=1)

In [ ]:
results_df = pd.DataFrame(columns=['Audio', 'Audio+User'])
results_df.index.astype(str, copy=False)
results_df.loc[0] = [user_audio_acc_mean.mean(),user_embeds_acc_mean.mean()]
results_df.loc[1] = [users_audio_recall_mean.mean(),user_embeds_recall_mean.mean()]
results_df.loc[2] = [user_audio_prec_mean.mean(),user_embeds_prec_mean.mean()]
results_df.loc[3] = [user_audio_f1_mean.mean(),user_embeds_f1_mean.mean()]
results_df.index = ['Accuracy', 'Recall', 'Precision', 'f1-score']
results_df = results_df.T
results_df